In [2]:
!pip install datasets transformers torch tokenizers huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# **Loading Data**

In [3]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import pandas as pd

dataset = load_dataset("SKNahin/bengali-transliteration-data")




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/300 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/333k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5006 [00:00<?, ? examples/s]

In [4]:
pd.DataFrame(dataset).head()

,train
0,{'bn': 'স্ক্রোল করে ২০/৩০ সেকেন্ড এর ভিডিও পান...
1,"{'bn': 'ও গুলা টরেন্ট সাইট এ পাবেন', 'rm': 'o ..."
2,{'bn': 'ভক্কর চক্কর পোস্ট একটা করলেই এপ্রুভড.…...
3,"{'bn': 'আমি টেস্ট করেই কোড দিছি…', 'rm': 'ami ..."
4,"{'bn': 'এতো কষ্টের কি আছে সাকিবওয়াপ.টক,সাকিবওয়..."


In [5]:
data = pd.DataFrame(dataset['train'])
data.head()

,bn,rm
0,স্ক্রোল করে ২০/৩০ সেকেন্ড এর ভিডিও পান নাই???,scroll kore 20/30 second er video pann nai???
1,ও গুলা টরেন্ট সাইট এ পাবেন,o gula Torrent site e paben
2,ভক্কর চক্কর পোস্ট একটা করলেই এপ্রুভড.… নিশ্চই ...,vokkor chokkor post akta korlei approved…. nis...
3,আমি টেস্ট করেই কোড দিছি…,ami test koreii code disi…
4,"এতো কষ্টের কি আছে সাকিবওয়াপ.টক,সাকিবওয়াপ.মল&এআ...","eto koster ki ache shakibwap.tk,shakibwap.ml&a..."


**Data Splitting**

In [6]:
from sklearn.model_selection import train_test_split

# Split into training and validation sets (80/20 split)
train_data, val_data= train_test_split(data, test_size=0.2, random_state=42)


print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")


Training samples: 4004
Validation samples: 1002


# **Data Understanding**

In [7]:
#Inspecting the data
train_data.head()

,bn,rm
831,মুভি দেখতে হবে,movie dekhte hobe
239,ভাইয়া আপনার অন্য একটা পোস্ট এ আমি কমেন্ট করেছি...,Via apner onno akta post a ami comment koreci ...
4687,ব্রো আমি পার্ট বাই পার্ট উইথ অল ডিটেিইলস পোস্ট...,Bro Ami part by part with all details post ban...
999,নোটিফিকেশন বার এ টাইম এর গড়মিল আছে,notification bar e time er gormil ache
4775,দিতে পারবো,dite parbo


In [8]:
train_data.isnull().sum()

,0
bn,0
rm,0


**Conclusion: Contains the text in Bangla and its Banglish rm version in 2 columns , target is to convert Banglish to Bangla using a model. There are no null values**

# **Data Preprocessing**

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")  # Using T5

def tokenize_data(df, tokenizer, max_length=128):
    inputs = list(df["rm"])
    targets = list(df["bn"])

    # Tokenize inputs and targets
    tokenized_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt")
    tokenized_targets = tokenizer(targets, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt")

    # Return dictionary
    return {
        "input_ids": tokenized_inputs["input_ids"],
        "attention_mask": tokenized_inputs["attention_mask"],
        "labels": tokenized_targets["input_ids"],
    }

train_encodings = tokenize_data(train_data, tokenizer)
val_encodings = tokenize_data(val_data, tokenizer)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

# **Model Selection**

In [10]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("t5-small")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

**Training the model**

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader

class BanglishToBanglaDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {key: tensor[idx] for key, tensor in self.encodings.items()}

train_dataset = BanglishToBanglaDataset(train_encodings)
val_dataset = BanglishToBanglaDataset(val_encodings)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)


**Defining the optimizer and rate scheduler**

In [12]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR

optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = StepLR(optimizer, step_size=1, gamma=0.95)


In [13]:
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def train_model(model, train_loader, val_loader, optimizer, scheduler, num_epochs=3):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}")
        model.train()
        train_loss = 0

        # Training loop
        for batch in tqdm(train_loader):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        print(f"Training loss: {train_loss / len(train_loader)}")

        # Validation loop
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                val_loss += outputs.loss.item()

        print(f"Validation loss: {val_loss / len(val_loader)}")
        scheduler.step()


In [ ]:
train_model(model, train_loader, val_loader, optimizer, scheduler, num_epochs=5)


Epoch 1


  5%|▌         | 13/251 [04:14<1:09:25, 17.50s/it]

In [ ]:
#Save model Locally
model.save_pretrained("./banglish-to-bangla-model")
tokenizer.save_pretrained("./banglish-to-bangla-model")


**TEST THE MODEL**

# **NOTE: ATLEAST 12HRS TIME WAS NEEDED, WE COULDVE SOLVED THE ENTIRE THING IF MORE TIME WAS GIVEN**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Path to the saved model directory
model_path = "./banglish-to-bangla-model"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

print("Model and tokenizer loaded successfully!")


In [ ]:
# Example Banglish input
banglish_input = "apnar naam ki"  # Replace with your own test case

# Tokenize the input
input_ids = tokenizer(banglish_input, return_tensors="pt").input_ids

# Generate the output
output_ids = model.generate(input_ids, max_length=50)

# Decode the output to get Bengali text
bengali_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Banglish Input:", banglish_input)
print("Bengali Output:", bengali_output)
